# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [84]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy import stats

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [6]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [179]:
# Your code here: Vemos le data base
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [13]:
# Your code here: vemos nulos o faltas de datos
salaries.isnull().sum().head()

Name                 0
Job Titles           0
Department           0
Full or Part-Time    0
Salary or Hourly     0
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [57]:
# Your code here:
dfSaralariohoras = salaries['Hourly Rate'].dropna()

dfSalariosAnualy = salaries['Annual Salary'].dropna()
print(len(dfSaralariohoras),len(dfSalariosAnualy))

8022 25161


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [38]:
# Your code here:df[['col1', 'col2', 'col3', 'col4']].groupby(['col1', 'col2']).agg(['mean', 'count'])
salariesDepartSum = salaries['Department'].value_counts()
salariesDepartSum.head()

POLICE           13414
FIRE              4641
STREETS & SAN     2198
OEMC              2102
WATER MGMNT       1879
Name: Department, dtype: int64

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [77]:
# Your code here: poner comentarios estoy valorando que hipótesis si rechazamos o no nula 0.004e94

#Hacemos la hipótesis nula de que cobran $30/h , como el PValue es muy bajo la hipótesis no se cumple.
#Por fa explicame por que el PValue es tan bajo y que significa

from scipy.stats import t

checkPValue = lambda p,threshold=0.05: "--> Null(H0) hypotesis rejected" if p < threshold else "--> We cannot reject the null hypotesis"


threshold=0.05

resA = stats.ttest_1samp(dfSaralariohoras,30)


print(checkPValue(resA[1]))
print(resA)

--> Null(H0) hypotesis rejected
Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92)


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [81]:
# Your code here:   Buscamos rechazar la hipótesis de que los sueldo son superiores a 86000$
resB = stats.ttest_1samp(dfSalariosAnualy,86000)

print(checkPValue(resB[1]))

print(resB)
#Rechazamos la hipótesis 

--> Null(H0) hypotesis rejected
Ttest_1sampResult(statistic=5.932870515690814, pvalue=3.015758519916386e-09)


Using the `crosstab` function, find the department that has the most hourly workers. 

In [93]:
# Your code here:
salariosCros = pd.crosstab(index=salaries["Department"],columns =salaries["Typical Hours"])
salariosCros.head()

Typical Hours,10.0,20.0,35.0,40.0
Department,,,,
ANIMAL CONTRL,1,18,0,0
AVIATION,0,18,15,1049
BUDGET & MGMT,0,0,2,0
BUSINESS AFFAIRS,0,7,0,0
CITY COUNCIL,7,36,18,3


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [ ]:
(salariesDepartSum.sort_values())# veo que el mayor numero de empleados son Police

In [118]:
# Your code here: Hipótesis de negar que los trabajadores tienen menos de 35/hora con más trabajadores
#uno los columnas de police y el idex que sea Hourly Rate
salariosMas = pd.crosstab(index=salaries["Hourly Rate"],columns =salaries["Department"])
#Seleciono solo a los policias elimino los nulos
policiasconsueldoporhoras =salariosMas["POLICE"].dropna()

#Hago la hipótesis nula para ver si tienen razón con que ganan menos de 35$/hora
respolicias= stats.ttest_1samp(policiasconsueldoporhoras,35)
print(checkPValue(resB[1]),    respolicias)
#Rechazo la hipotesis.

--> Null(H0) hypotesis rejected Ttest_1sampResult(statistic=-811.4281122281091, pvalue=1.305155248942784e-300)


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [144]:
#Calculo el error estadart
errorstandart = stats.sem(salaries["Hourly Rate"],  ddof=-1, nan_policy="omit")
#Quito los valores nulos a la columna salaries["Hourly Rate"]
salarioshora = salaries["Hourly Rate"].dropna()
#Calculo la media de salarios
mean = np.mean(salarioshora)
#Calculo el intervalo de confidence
confidenceinterval = t.interval(alpha = 0.95, df=salarioshora ,loc =mean, scale=errorstandart)
#Hago la media del intervalo de confianza
resultado = np.mean(confidenceinterval)
print(resultado)

32.78855771628024


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [170]:
# Your code here:
#Calculo el error estadart
errorstandart2 = stats.sem(salaries["Annual Salary"] ,  ddof=-1, nan_policy="omit")
#Quito los valores nulos a la columna 
salarioshora2 = salaries["Annual Salary"].dropna()
#Calculo la media de salarios
mean2 = np.mean(salarioshora2)

#Calculo el intervalo de confidence
confidenceinterval = t.interval(alpha = 0.95, df=salarioshora2 ,loc =mean2, scale=errorstandart2)
#Hago la media del intervalo de confianza
resultado2 = np.mean(confidenceinterval)
print(resultado2)
#No entiendo este resultado si antes salia que no tenian el sueldo de 86000

86786.99979015143


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [186]:
import statsmodels
from statsmodels.stats.proportion import proportions_ztest

# Your code here: statsmodels.stats.proportion.proportions_ztest
#(count, nobs, value=None, alternative='two-sided', prop_var=False
# Hipótesis nula vamos a decir que nen la ciudad de Chicago no tienen un 25% diferente del coeficiente de confianza
salaries
count = len(salaries["Hourly Rate"])
nobs = 1
value = 0.25
print(count)
stat, pval = proportions_ztest(count, nobs, value,prop_var=0)
print(stat,pval)
#Ni idea de como funciona no lo entiendo :(

33183
nan nan
